<a href="https://colab.research.google.com/github/jesusGS01/TGINE/blob/main/6_1_BERT_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sesión 6.1 BERT/RoBERTa y contextual word embeddings

En esta sesión se muestra un ejemplo de funcionamiento de los word embeddings contextuales de los Transformers como BERT o RoBERTa.

Primero instalaremos la libería de transformers y de datasets

In [ ]:
# Install libraries
!pip3 install -U transformers datasets

## Apartado 1.1 Descargamos el modelo de BERT preentrenado

Descargamos un modelo de BERT preentrenado como BETO o mBERT.

Cada word embedding de BERT está representado por vectores de 768 características para los modelos *base* y 1024 en los modelos *large*.

In [2]:
import transformers

# Modelo de BETO
#path_bert_model = 'dccuchile/bert-base-spanish-wwm-uncased'
# Modelo de multilingual BERT
#path_bert_model = 'bert-base-multilingual-cased'
# Modelo de distilbert en español
path_bert_model = 'CenIA/distillbert-base-spanish-uncased'
# Modelo de RoBERTa de MarIA
path_roberta_model = 'PlanTL-GOB-ES/roberta-base-bne'

# Obtenemos el tokenizer y el modelo de BERT
tokenizer = transformers.AutoTokenizer.from_pretrained (path_bert_model)
bert_model = transformers.AutoModel.from_pretrained (path_bert_model, output_hidden_states=True)

# Obtenemos el tokenizer y el modelo de RoBERTa
roberta_tokenizer = transformers.AutoTokenizer.from_pretrained (path_roberta_model)
roberta_model = transformers.AutoModel.from_pretrained (path_roberta_model, output_hidden_states=True)


Some weights of RobertaModel were not initialized from the model checkpoint at PlanTL-GOB-ES/roberta-base-bne and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Apartado 1.2 WordPiece tokenizer de BERT

BERT usa lo que se llama un WordPieceTokenizer que trabaja dividiendo palabras en distintos "trozos" o tokens para tener en cuenta las inflexiones de las palabras como sufijos y conjugaciones verbales.

Los tokens especiales que utiliza BERT son los siguientes:
* [UNK] – The unknown token. Un token que no está en el vocabulario que no puede convertirse a ningún id y se usa este token para eso.
* [SEP] – The separator token. Cuando se quieren separar secuencias de texto para tareas de clasificación ode pregunta-respuesta.
* [PAD] – The token used for padding. El token que se utiliza de relleno para textos de distinto tamaño.
* [CLS] – The classifier token. Se usa para la clasificación del fragmento de texto. Es el primer token del texto.
* [MASK] – The token used for masking values. Este token se utiliza cuando se quiere entrenar con modelado del lenguaje con mask. Este token es el qeu se quiere predecir.

In [3]:
text = "Estudiaré la asignatura de tecnologías de gestión de información no estructurada y terminaré las prácticas en casa."
tokenizer.tokenize(text, add_special_tokens=True)

['[CLS]',
 'estudiar',
 '##é',
 'la',
 'asigna',
 '##tura',
 'de',
 'tecnologías',
 'de',
 'gestión',
 'de',
 'información',
 'no',
 'estructura',
 '##da',
 'y',
 'terminar',
 '##é',
 'las',
 'prácticas',
 'en',
 'casa',
 '.',
 '[SEP]']

##Apartado 1.3 BPE tokenizer de RoBERTA
Los modelos RoBERTa por otro lado utilizan un tokenizer llamad BPE (Byte-Pair Encoding).
Los tokens especiales que utiliza RoBERTa son los siguientes:
* \<s> Para indicar el inicio de la frase
* \</s> Para indicar el fin de la frase

In [4]:
text = "Estudiaré la asignatura de tecnologías de gestión de información no estructurada y terminaré las prácticas en casa."
roberta_tokenizer.tokenize(text, add_special_tokens=True)

['<s>',
 'Estudi',
 'arÃ©',
 'Ġla',
 'Ġasignatura',
 'Ġde',
 'ĠtecnologÃŃas',
 'Ġde',
 'ĠgestiÃ³n',
 'Ġde',
 'ĠinformaciÃ³n',
 'Ġno',
 'Ġestructurada',
 'Ġy',
 'Ġterminar',
 'Ã©',
 'Ġlas',
 'ĠprÃ¡cticas',
 'Ġen',
 'Ġcasa',
 '.',
 '</s>']

## Apartado 1.4 Word embeddings contextuales de BERT

BERT tiene distintas capas ocultas dependiendo del modelo *base* o *large* preentrenado. En los modelos *base* el número de capas son 12 y en los modelos *large* este número es 24 que junto con la capa de entrada forman 13 y 25 capas respectivamente.

Para cada token de nuestra frase tendremos entonces 13 capas de vectores de 768 características.

Los embeddings de cada Token son distintos y dependen del contexto donde aparezcan en el texto. En el código siguiente se muestra un ejemplo de distintos vectores de la misma palabra "naranja" según su contexto. Para el cálculo del word embedding de la palabra utilizamos las 4 últimas capas y sumamos sus valores.

In [5]:
 import numpy as np
 import torch
 from scipy import spatial

 from transformers import AutoTokenizer, AutoModel

 def get_hidden_states(encoded, token_ids_word, model):
     """Push input IDs through model. Stack and sum `layers` (last four by default).
        Select only those subword token outputs that belong to our word of interest
        and average them."""
     with torch.no_grad():
         output = model(**encoded)
     # Get all hidden states
     states = output.hidden_states
     # Stack and sum the four last layers
     output = torch.stack(states[-4:]).sum(0).squeeze()
     # Only select the tokens that constitute the requested word
     word_tokens_output = output[token_ids_word]

     return word_tokens_output.mean(dim=0)

 # Esta función obtiene el vector del token que se encuentra en la posición idx de la frase.
 def get_word_vector(sent, idx, tokenizer, model):
     """Get a word vector by first tokenizing the input sentence, getting all token idxs
        that make up the word of interest, and then `get_hidden_states`."""
     encoded = tokenizer.encode_plus(sent, return_tensors="pt", add_special_tokens=False)
     # get all token idxs that belong to the word of interest
     token_ids_word = np.where(np.array(encoded.word_ids()) == idx)

     return get_hidden_states(encoded, token_ids_word, model)


In [6]:
text1 = "La camiseta que tengo es de color naranja y hace que se resalten mis ojos."
token = 'naranja'
# Obtenemos el id del token
idx1=tokenizer.tokenize(text1).index(token)

# Obtenemos la representación vectorial de token usando la función definida anteriormente
word_embedding1 = get_word_vector(text1, idx1, tokenizer, bert_model)
# imprimimos ese vector del token
print('El vector del token ',token,' es: ', word_embedding1)

El vector del token  naranja  es:  tensor([-5.6051e-01, -6.3185e-02, -5.9235e-01,  1.5971e+00, -6.5679e-01,
        -6.1817e-01, -7.3410e-01, -2.2324e-01,  3.9488e-01,  1.9866e-01,
         9.7899e-01, -1.5964e+00, -8.5425e-02, -7.0487e-03, -2.1820e-01,
         1.4673e+00, -1.9097e-01, -1.9395e+00, -9.8395e-01,  6.6020e-01,
         1.1358e+00,  1.1473e+00, -1.9881e+00,  5.4002e-01, -1.4425e+00,
        -2.8578e-01,  1.2786e+00, -3.4850e+00, -1.1959e+00,  2.7442e-01,
         6.7153e-01, -6.1663e-01, -1.4963e+00,  6.1953e-01, -2.7531e+00,
         1.8823e+00, -1.7678e+00,  1.7454e+00, -2.9325e+00,  3.4799e+00,
         1.2284e+00,  1.8303e+00, -1.6882e+00, -6.7536e-01, -3.5000e-02,
        -2.0438e+00,  2.5104e-01, -5.5653e-01,  7.5502e-01,  1.9775e+00,
         3.6566e-01,  6.1477e-01, -1.4052e+00,  5.6690e-01, -1.3444e+00,
        -2.7857e-01,  3.7521e+00,  1.3042e-01, -6.8093e-01, -1.5953e+00,
        -1.1882e+00,  2.3686e+00, -4.7112e-01, -2.4560e-01,  1.9825e+00,
        -6.8667e

In [7]:
text2 = "La fruta que más me gusta es la naranja y después el pomelo."
idx2=tokenizer.tokenize(text2).index(token)
word_embedding2 = get_word_vector(text2, idx2, tokenizer, bert_model)

text3 = "Lo que tienes que hacer es comer la naranja y el plátano."
idx3=tokenizer.tokenize(text3).index(token)
word_embedding3 = get_word_vector(text3, idx3, tokenizer, bert_model)

text4 = "Pinta la valla de naranja y el tejado de rojo."
idx4=tokenizer.tokenize(text4).index(token)
word_embedding4 = get_word_vector(text4, idx4, tokenizer, bert_model)

# Imprimimos los textos para después ver las similitudes
print("1: ",text1)
print("2: ",text2)
print("3: ",text3)
print("4: ",text4)

# Calculamos las similitudes entre los tokens en los distintos textos
similarity1_2 = 1 - spatial.distance.cosine (word_embedding1, word_embedding2)
print('La similitud entre el primer y segundo texto para el token ',token,' es:',similarity1_2)
similarity1_3 = 1 - spatial.distance.cosine (word_embedding1, word_embedding3)
print('La similitud entre el primer y tercer texto para el token ',token,' es:',similarity1_3)
similarity1_4 = 1 - spatial.distance.cosine (word_embedding1, word_embedding4)
print('La similitud entre el primer y cuarto texto para el token ',token,' es:',similarity1_4)
similarity2_3 = 1 - spatial.distance.cosine (word_embedding2, word_embedding3)
print('La similitud entre el segundo y tercer texto para el token ',token,' es:',similarity2_3)
similarity2_4 = 1 - spatial.distance.cosine (word_embedding2, word_embedding4)
print('La similitud entre el segundo y cuarto texto para el token ',token,' es:',similarity2_4)
similarity3_4 = 1 - spatial.distance.cosine (word_embedding3, word_embedding4)
print('La similitud entre el tercer y cuarto texto para el token ',token,' es:',similarity3_4)

1:  La camiseta que tengo es de color naranja y hace que se resalten mis ojos.
2:  La fruta que más me gusta es la naranja y después el pomelo.
3:  Lo que tienes que hacer es comer la naranja y el plátano.
4:  Pinta la valla de naranja y el tejado de rojo.
La similitud entre el primer y segundo texto para el token  naranja  es: 0.825275182723999
La similitud entre el primer y tercer texto para el token  naranja  es: 0.7932690978050232
La similitud entre el primer y cuarto texto para el token  naranja  es: 0.8635101914405823
La similitud entre el segundo y tercer texto para el token  naranja  es: 0.9415081739425659
La similitud entre el segundo y cuarto texto para el token  naranja  es: 0.8137462139129639
La similitud entre el tercer y cuarto texto para el token  naranja  es: 0.8238715529441833


Imprimimos los tokens y los ids de los tokens asignados por el tokenizer para el texto 1

In [8]:
tokens = tokenizer.tokenize(text1)
print(tokens)
encoded = tokenizer.encode_plus(text1, return_tensors="pt", add_special_tokens=False)
print(encoded.input_ids)

['la', 'camiseta', 'que', 'tengo', 'es', 'de', 'color', 'naranja', 'y', 'hace', 'que', 'se', 'resal', '##ten', 'mis', 'ojos', '.']
tensor([[ 1032, 14156,  1041,  1484,  1028,  1009,  3614, 13177,  1040,  1647,
          1041,  1057, 13758,  1181,  1285,  3154,  1008]])


## Apartado 1.4 Word embeddings contextuales de RoBERTA


In [9]:
text1 = "La camiseta que tengo es de color naranja y hace que se resalten mis ojos."
token = 'Ġnaranja'

# Obtenemos el id del token
idx1=roberta_tokenizer.tokenize(text1).index(token)

# Obtenemos la representación vectorial de token usando la función definida anteriormente
word_embedding1 = get_word_vector(text1, idx1, roberta_tokenizer, roberta_model)

text2 = "La fruta que más me gusta es la naranja y después el pomelo."
idx2=roberta_tokenizer.tokenize(text2).index(token)
word_embedding2 = get_word_vector(text2, idx2, roberta_tokenizer, roberta_model)

text3 = "Lo que tienes que hacer es comer la naranja y el plátano."
idx3=roberta_tokenizer.tokenize(text3).index(token)
word_embedding3 = get_word_vector(text3, idx3, roberta_tokenizer, roberta_model)

text4 = "Pinta la valla de naranja y el tejado de rojo."
idx4=roberta_tokenizer.tokenize(text4).index(token)
word_embedding4 = get_word_vector(text4, idx4, roberta_tokenizer, roberta_model)

# Imprimimos los textos para después ver las similitudes
print("1: ",text1)
print("2: ",text2)
print("3: ",text3)
print("4: ",text4)

# Calculamos las similitudes entre los tokens en los distintos textos
similarity1_2 = 1 - spatial.distance.cosine (word_embedding1, word_embedding2)
print('La similitud entre el primer y segundo texto para el token ',token,' es:',similarity1_2)
similarity1_3 = 1 - spatial.distance.cosine (word_embedding1, word_embedding3)
print('La similitud entre el primer y tercer texto para el token ',token,' es:',similarity1_3)
similarity1_4 = 1 - spatial.distance.cosine (word_embedding1, word_embedding4)
print('La similitud entre el primer y cuarto texto para el token ',token,' es:',similarity1_4)
similarity2_3 = 1 - spatial.distance.cosine (word_embedding2, word_embedding3)
print('La similitud entre el segundo y tercer texto para el token ',token,' es:',similarity2_3)
similarity2_4 = 1 - spatial.distance.cosine (word_embedding2, word_embedding4)
print('La similitud entre el segundo y cuarto texto para el token ',token,' es:',similarity2_4)
similarity3_4 = 1 - spatial.distance.cosine (word_embedding3, word_embedding4)
print('La similitud entre el tercer y cuarto texto para el token ',token,' es:',similarity3_4)

1:  La camiseta que tengo es de color naranja y hace que se resalten mis ojos.
2:  La fruta que más me gusta es la naranja y después el pomelo.
3:  Lo que tienes que hacer es comer la naranja y el plátano.
4:  Pinta la valla de naranja y el tejado de rojo.
La similitud entre el primer y segundo texto para el token  Ġnaranja  es: 0.8016127347946167
La similitud entre el primer y tercer texto para el token  Ġnaranja  es: 0.7683536410331726
La similitud entre el primer y cuarto texto para el token  Ġnaranja  es: 0.5195296406745911
La similitud entre el segundo y tercer texto para el token  Ġnaranja  es: 0.9388827681541443
La similitud entre el segundo y cuarto texto para el token  Ġnaranja  es: 0.5220345258712769
La similitud entre el tercer y cuarto texto para el token  Ġnaranja  es: 0.522158682346344


## Apartado 1.5 Bert sentence embeddigns
Se puede calcular también el vector que representa toda el texto usando los sentence embeddings.

Para eso debemos instalar la libraría "sentence-transformers" de SBERT.net

Aunque se pueden utilizar los modelos de BERT y Roberta preentrenados, hay algunos otros modelos que se han entrenado para mejorar la similitud semántica de esos embeddings (Pretrained Models disponibles en https://www.sbert.net/docs/pretrained_models.html)

In [10]:
# Instalamos primero la librería necesaria
!pip3 install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.1 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=f3742672aea7b8a7516cb825cb21ffd7c8df2ddc134e7bdc4443b52713b2dcf3
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


Probamos los BERT Sentence embeddins para calcular la similitud ente distintos textos como hicimos en la sesión anterior.

In [11]:
from sentence_transformers import SentenceTransformer, models
from torch import nn
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
# Definimos un conjunto de textos
textos=['El procesamiento del lenguaje natural (PLN o NLP) es un campo dentro de la inteligencia artificial y la lingüística aplicada que estudia las interacciones mediante uso del lenguaje natural entre los seres humanos y las máquinas. \
Más concretamente se centra en el procesamiento de las comunicaciones humanas, dividiéndolas en partes, e identificando los elementos más relevantes del mensaje.\
Con la Comprensión y Generación de Lenguaje Natural, busca que las máquinas consigan entender, interpretar y manipular el lenguaje humano.'
, 'El procesamiento del lenguaje natural (NLP, por sus siglas en inglés) es una rama de la inteligencia artificial que ayuda a las computadoras a entender, interpretar y manipular el lenguaje humano. \
NLP toma elementos prestados de muchas disciplinas, incluyendo la ciencia de la computación y la lingüística computacional, en su afán por cerrar la brecha entre la comunicación humana y el entendimiento de las computadoras."""], """El procesamiento del lenguaje natural (PLN o NLP) es un campo dentro de la inteligencia artificial y la lingüística aplicada que estudia las interacciones mediante uso del lenguaje natural entre los seres humanos y las máquinas. Más concretamente se centra en el procesamiento de las comunicaciones humanas, dividiéndolas en partes, e identificando los elementos más relevantes del mensaje. Con la Comprensión y Generación de Lenguaje Natural, busca que las máquinas consigan entender, interpretar y manipular el lenguaje humano.'
, 'La lingüística computacional es un campo interdisciplinario que se ocupa del desarrollo de formalismos del funcionamiento del lenguaje natural, tales que puedan ser transformados en programas ejecutables para un ordenador. \
Dicho desarrollo se sitúa entre el modelado basado en reglas y el modelado estadístico del lenguaje natural desde una perspectiva computacional, y en él participan lingüistas e informáticos especializados en inteligencia artificial, psicólogos cognoscitivos y expertos en lógica, entre otros.'
, 'El aprendizaje automático es un tipo de inteligencia artificial (AI) que proporciona a las computadoras la capacidad de aprender, sin ser programadas explícitamente. El aprendizaje automático se centra en el desarrollo de programas informáticos que pueden cambiar cuando se exponen a nuevos datos.'
, 'El  aprendizaje profundo es un tema que cada vez adquiere mayor relevancia en el campo de la inteligencia artificial (IA). Siendo una subcategoría del aprendizaje automático, el aprendizaje profundo trata del uso de redes neuronales para mejorar cosas tales como el reconocimiento de voz, la visión por ordenador y el procesamiento del lenguaje natural. \
Rápidamente se está convirtiendo en uno de los campos más solicitados en informática. \
En los últimos años, el aprendizaje profundo ha ayudado a lograr avances en áreas tan diversas como la percepción de objetos, el procesamiento del lenguaje natural y el reconocimiento de voz (todas ellas áreas especialmente complejas para los investigadores en IA).',
'El coste de la energía va a subir mucho los próximos meses y la población va a tener que pagar cantidades excesivas a las eléctricas']

# Calculamos la similitud usando sentence embeddings
word_embedding_model = models.Transformer(path_bert_model, max_seq_length=768)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
dense_model = models.Dense(in_features=pooling_model.get_sentence_embedding_dimension(), out_features=768, activation_function=nn.Tanh())

model = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model])

embeddings = model.encode(textos)
print(embeddings)

results = cosine_similarity(embeddings[1::],embeddings[0].reshape(1,-1)).reshape(-1,) # Op -- (n_docs,1) -- Cosine Sim with each doc
print('\n',results)


[[-0.07933853 -0.39736724  0.23490235 ... -0.08782886 -0.21158086
   0.30048597]
 [-0.10962841 -0.38395476  0.24338022 ... -0.18215805 -0.23653388
   0.3314694 ]
 [-0.17937563 -0.41728777  0.15986998 ... -0.11907891 -0.21189739
   0.30312148]
 [-0.11101843 -0.4735786   0.1275042  ... -0.13632536 -0.17866266
   0.3943588 ]
 [-0.1014069  -0.35934368  0.17549595 ... -0.1394456  -0.25048938
   0.3956558 ]
 [ 0.00534249 -0.07229532 -0.10286631 ... -0.14552638  0.07034707
   0.17346674]]

 [0.97796583 0.93321955 0.8875055  0.93860763 0.6302492 ]


Probamos a entrenar el clasificador de tuits de prácticas anteriores

In [12]:
# Descargamos el fichero de datasetEspañol.csv
!wget http://dis.um.es/~valencia/recursosTGINE/datasetEspañol.csv


--2023-11-15 15:42:43--  http://dis.um.es/~valencia/recursosTGINE/datasetEspa%C3%B1ol.csv
Resolving dis.um.es (dis.um.es)... 155.54.239.5
Connecting to dis.um.es (dis.um.es)|155.54.239.5|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1953117 (1.9M) [text/csv]
Saving to: ‘datasetEspañol.csv’

datasetEspañol.csv  100%[===================>]   1.86M  1.99MB/s    in 0.9s    

2023-11-15 15:42:44 (1.99 MB/s) - ‘datasetEspañol.csv’ saved [1953117/1953117]



Generamos los sentences embeddings de los tuits de entrenamiento y validación.
Este proceso tarda mucho si se ejectua en CPU. Por favor, acordaos de usar un entorno con GPU.

In [13]:
import json
import csv

import pandas
from sklearn.svm import LinearSVC
df = pandas.read_csv("datasetEspañol.csv",encoding="UTF-8")

p_train = 0.80 # Porcentaje de train.
p_test = 0.20 # Porcentaje de train.

from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size = p_test)

# Ponemos en lower_case los dos conjuntos de tweets
df_train.tweet = df_train.tweet.apply(lambda x: x.lower())
df_test.tweet = df_test.tweet.apply(lambda x: x.lower())

sentence_train = model.encode(df_train.tweet.tolist())
sentence_test = model.encode((df_test.tweet.tolist()))

Entrenamos el modelo SVM que hemos usado en prácticas anteriores

In [14]:
# Entrenamos el mismo SVN que en sesiones anteriores
clf_sentence_embeddings = LinearSVC(random_state=0, tol=1e-5).fit(sentence_train, df_train.label)
predicted = clf_sentence_embeddings.predict(sentence_test)

accuracy = np.mean(predicted == df_test.label)
print("Resultados Sentence Embeddings ----- Accuracy:", accuracy)
from sklearn import metrics
print(metrics.classification_report(df_test.label, predicted))

Resultados Sentence Embeddings ----- Accuracy: 0.8691275167785235
              precision    recall  f1-score   support

    negative       0.80      0.73      0.76       346
    positive       0.89      0.93      0.91       846

    accuracy                           0.87      1192
   macro avg       0.85      0.83      0.84      1192
weighted avg       0.87      0.87      0.87      1192



Probamos algunos ejemplos de inferencia con el modelo ya entrenado

In [15]:
# Probamos algunos ejemplos con los modelos inferidos
textos = ['hay muchos más muertos por covid',
          'el número de afectados por covid aumenta',
          'vamos a salir de la pandemia',
          'ánimo a todos'
]
# Codificamos estos documentos
textos_SE = model.encode(textos)
# Predecimos
predicted = clf_sentence_embeddings.predict(textos_SE)

# Imprimimos los textos y su predicción para TF
for doc, category_tf in zip(textos, predicted):
  print('TF: %r => %s' % (doc, category_tf))

TF: 'hay muchos más muertos por covid' => negative
TF: 'el número de afectados por covid aumenta' => negative
TF: 'vamos a salir de la pandemia' => positive
TF: 'ánimo a todos' => positive
